# gBizINFOの法人データを用いたスタートアップ企業の分析

https://info.gbiz.go.jp/hojin/swagger-ui/index.html#/gBizINFO%20REST%20API/searchInfo に記載されている内容を参考にして、gBizInfo のAPI を用いてデータを取得します。

今回は、2019年から2023年に設立された企業情報を取得します。

In [1]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path

# データ取得に用いるライブラリ
from urllib.parse import urljoin
import requests
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET 

# データ操作に用いるライブラリ
import time
import pandas as pd

current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch12").resolve()

In [3]:
# 取得したアクセストークンを指定
headers_gbiz = {
    "Accept": "application/json",
    "X-hojinInfo-api-token": api_token
}
url_hojin = "https://info.gbiz.go.jp/hojin/v1/hojin"

founded_year = ",".join(str(n) for n in range(2019, 2024))

params_gbiz = {
    "founded_year": founded_year,
    "limit": "5000"
}
res_hojin = requests.get(url_hojin, params=params_gbiz, headers=headers_gbiz)
out = res_hojin.json()

gbiz_registered_firm = pd.json_normalize(out, record_path=["hojin-infos"])

In [4]:
gbiz_registered_firm.head()

,corporate_number,postal_code,location,name,status,number_of_activity,update_date,name_en
0,1010001197825,1020084,東京都千代田区二番町１－２,株式会社ＪＢＳ,-,2,2019-01-28T00:00:00+09:00,NaN
1,1010001198690,1010065,東京都千代田区西神田２丁目７－６川合ビル２７号室,株式会社ラス・マリアナス,-,1,2022-01-14T00:00:00+09:00,NaN
2,1010001200092,1020083,東京都千代田区麹町４丁目４番地３ピネックス麹町６階,モルゲンロット株式会社,-,8,2023-09-19T00:00:00+09:00,NaN
3,1010001201198,1040031,東京都中央区京橋３丁目１０番１号ＮＥＷＳ京橋８階８０１号室,テクネティクス・グループ・ジャパン株式会社,-,2,2019-08-29T00:00:00+09:00,NaN
4,1010001201388,1020083,東京都千代田区麹町３丁目５番地４麹町インテリジェントビルＢ－１,株式会社タストＡｌｐｈａ,-,4,2024-11-06T00:00:00+09:00,NaN


続いて、先程取得した法人番号を用いて、企業のその他の情報を取得します。

In [5]:
# 各法人のDataFrameを一時的に格納するためのリスト
hojin_info_df = []

for corp_num in tqdm(gbiz_registered_firm.corporate_number, desc="法人情報取得中"):
    url3 = f"https://info.gbiz.go.jp/hojin/v1/hojin/{corp_num}"
    try:
        response = requests.get(url3, params=params_gbiz, headers=headers_gbiz)
        response.raise_for_status()  # HTTPエラーのチェック

        temp_df = pd.json_normalize(response.json()["hojin-infos"])
        
        # DataFrameをリストに追加
        hojin_info_df.append(temp_df)

    except requests.exceptions.RequestException as e:
        print(f"法人番号 {corp_num} の取得に失敗: {e}")
    except KeyError:
        # "hojin-infos" というキーがJSON内に存在しない場合のエラー
        print(f"法人番号 {corp_num} のレスポンス形式が正しくありません。")

# ループ完了後、リストに格納された全てのDataFrameを一度に結合する
if hojin_info_df:
    gbiz_houjin_firm = pd.concat(hojin_info_df, ignore_index=True)
else:
    gbiz_houjin_firm = pd.DataFrame() # 1件も成功しなかった場合

法人情報取得中:   0%|          | 0/5000 [00:00<?, ?it/s]

データの中身を確認します。代表取締役の名前や、住所情報などが取得出来ていることが確認出来ます。

In [6]:
gbiz_houjin_firm

,corporate_number,representative_position,representative_name,postal_code,location,name,kana,status,business_items,qualification_grade,...,employee_number,company_size_male,company_size_female,business_summary,company_url,close_date,close_cause,name_en,founding_year,capital_stock
0,1010001197825,代表取締役,中鉢 真輔,1020084,東京都千代田区二番町１－２,株式会社ＪＢＳ,じぇいびーえす,-,"[229, 307, 310, 315]",、D、D、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1010001198690,代表取締役,北村 敦,1010065,東京都千代田区西神田２丁目７－６川合ビル２７号室,株式会社ラス・マリアナス,らすまりあなす,-,"[214, 221, 229]",、D、、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1010001200092,代表取締役,森本 竜英,1020083,東京都千代田区麹町４丁目４番地３ピネックス麹町６階,モルゲンロット株式会社,もるげんろっと,-,"[216, 217, 303, 304, 306]",、C、C、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1010001201198,代表取締役,ブラッドリー・トーマス・ロッジ,1040031,東京都中央区京橋３丁目１０番１号ＮＥＷＳ京橋８階８０１号室,テクネティクス・グループ・ジャパン株式会社,てくねてぃくすぐるーぷじゃぱん,-,"[204, 212, 215, 218, 221, 228, 302]",、C、C、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010001201388,代表取締役,大久保 卓磨,1020083,東京都千代田区麹町３丁目５番地４麹町インテリジェントビルＢ－１,株式会社タストＡｌｐｈａ,たすとあるふぁ,-,"[301, 302, 303, 304, 306, 315]",、、D、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,9180001142609,代表取締役,岩崎 藍,4510031,愛知県名古屋市西区城西５丁目２７番１号,株式会社中京電力,ちゅうきょうでんりょく,-,"[213, 315]",、C、C、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,9180001142641,代表取締役,百済 まさみ,4600011,愛知県名古屋市中区大須３丁目３０－６０ＯＳＵ３０１ビル６Ｆ,株式会社Ｃａｎｓ,きゃんず,-,"[201, 202, 203, 204, 205, 206, 207, 208, 209, ...",、D、D、C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,9180001145165,代表取締役,中山 恵二,4500002,愛知県名古屋市中村区名駅４丁目２４番１６号広小路ガーデンアベニュー４階,リプラ株式会社,りぷら,-,"[301, 303, 304, 309, 315]",、、D、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,9180001145512,代表取締役,岩橋 知裕,4860963,愛知県春日井市春日井町字黒鉾３８番地３,株式会社Ｂｅｓｔ Ｔｅａｍ,べすとちーむ,-,"[301, 302, 303, 304, 313, 315]",、、D、,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


データ同士を接合します。

In [7]:
gbiz_merged_df = gbiz_registered_firm.merge(gbiz_houjin_firm, on='corporate_number' , suffixes=('', '_detail'))

In [8]:
gbiz_merged_df.head()

,corporate_number,postal_code,location,name,status,number_of_activity,update_date,name_en,representative_position,representative_name,...,employee_number,company_size_male,company_size_female,business_summary,company_url,close_date,close_cause,name_en_detail,founding_year,capital_stock
0,1010001197825,1020084,東京都千代田区二番町１－２,株式会社ＪＢＳ,-,2,2019-01-28T00:00:00+09:00,NaN,代表取締役,中鉢 真輔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1010001198690,1010065,東京都千代田区西神田２丁目７－６川合ビル２７号室,株式会社ラス・マリアナス,-,1,2022-01-14T00:00:00+09:00,NaN,代表取締役,北村 敦,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1010001200092,1020083,東京都千代田区麹町４丁目４番地３ピネックス麹町６階,モルゲンロット株式会社,-,8,2023-09-19T00:00:00+09:00,NaN,代表取締役,森本 竜英,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1010001201198,1040031,東京都中央区京橋３丁目１０番１号ＮＥＷＳ京橋８階８０１号室,テクネティクス・グループ・ジャパン株式会社,-,2,2019-08-29T00:00:00+09:00,NaN,代表取締役,ブラッドリー・トーマス・ロッジ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010001201388,1020083,東京都千代田区麹町３丁目５番地４麹町インテリジェントビルＢ－１,株式会社タストＡｌｐｈａ,-,4,2024-11-06T00:00:00+09:00,NaN,代表取締役,大久保 卓磨,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


API ではなくZIP からファイルを読み込む場合

In [16]:
csv_df = pd.read_csv(
data_path / "Kihonjoho_SJIS_20250921.zip",
encoding="shift_jis",
low_memory=False,
usecols=["法人番号", "法人名", "郵便番号", "本社所在地","ステータス", "設立年月日", "創業年", "最終更新日"],
parse_dates=["設立年月日", "最終更新日"],
)
trans_map = {"法人番号": "corporate_number", "法人名": "name","本社所在地": "location", "郵便番号": "postal_code","ステータス": "status", "最終更新日": "update_date"}
required_cols = list(trans_map.keys()) + ["founded_year"]
founded_year_combined = csv_df["設立年月日"].dt.year.fillna(csv_df["創業年"]).astype("Int64")
founded_from_2019 = (csv_df["設立年月日"].dt.year >= 2019) | (csv_df["創業年"] >= 2019)
founded_until_2023 = (csv_df["設立年月日"].dt.year < 2024) | (csv_df["創業年"] < 2024)
gbiz_merged_df = (csv_df
.assign(**{"founded_year": founded_year_combined})
.loc[csv_df["法人名"].notna() &
founded_from_2019 & founded_until_2023 &
(founded_year_combined.isna() | (founded_year_combined >= 2019)),
required_cols]
.rename(columns=trans_map)
)

In [19]:
gbiz_merged_df.head()

,corporate_number,name,location,postal_code,status,update_date,founded_year
584,3010701042353,株式会社キカリ,東京都千代田区麹町３丁目３番地８,1020083.0,NaN,2023-07-14 00:00:00+09:00,2022
1143,6010001207083,株式会社ＳＥＶＥＮ ＳＥＡＬＳ,神奈川県横浜市西区みなとみらい２丁目２番１号ランドマークタワー３１Ｆ,2200012.0,NaN,2023-07-14 00:00:00+09:00,2020
1257,6130005015768,一般社団法人相続なんでも相談センター,大阪府吹田市垂水町３丁目４番４号,5640062.0,NaN,2023-07-14 00:00:00+09:00,2019
7398,1430001078470,株式会社スクラッチシステムマネージメンツ,北海道札幌市中央区北三条西７丁目１番地１ＳＡＫＵＲＡ−Ｎ３,600003.0,NaN,2021-01-04 00:00:00+09:00,2019
7506,1430001079832,株式会社ザイマックス北海道,北海道札幌市中央区北一条西３丁目２番地,600001.0,NaN,2020-01-16 00:00:00+09:00,2019


さて、ここで都道府県ごとにどの程度スタートアップが存在するのかをテストしたいと思います。
ここでは、正規表現を用いて先ほどの住所情報から都道府県名を取り出すことにします。

In [20]:
firms_with_prefecture = gbiz_merged_df.assign(**{
    "都道府県": gbiz_merged_df["location"]
        .str.extract(r'^(東京都|北海道|(?:京都|大阪)府|.{2,3}県)', expand=False)
        .fillna("")
})

In [21]:
firms_with_prefecture["都道府県"].unique()[:5]

array(['東京都', '神奈川県', '大阪府', '北海道', '青森県'], dtype=object)

得られた都道府県の情報を用いて、過去4年に開業した企業の都道府県ごとの数について確認しましょう。

In [22]:
firms_with_prefecture.groupby('都道府県').size().sort_values(ascending=False)

都道府県
東京都     2159
大阪府      374
神奈川県     311
福岡県      200
愛知県      179
北海道      177
千葉県      134
埼玉県      134
沖縄県      126
兵庫県      118
茨城県       98
京都府       90
広島県       82
宮城県       69
静岡県       63
福島県       49
鹿児島県      46
長野県       44
岐阜県       40
岡山県       38
熊本県       37
新潟県       37
三重県       37
群馬県       36
山口県       30
奈良県       29
長崎県       29
青森県       28
石川県       26
香川県       25
滋賀県       25
福井県       25
宮崎県       25
栃木県       24
高知県       22
愛媛県       21
岩手県       20
徳島県       20
和歌山県      19
秋田県       17
大分県       16
富山県       16
山梨県       14
山形県       13
鳥取県       11
島根県       10
佐賀県        9
dtype: int64

やはり、東京都にスタートアップが多く存在することがわかります。

なお、これらのスタートアップがデータ取得の段階でも存在しているかどうかの確認をしたいと思います。ここでは、statusを用います。

In [23]:
firms_with_prefecture.groupby('status').size().sort_values(ascending=False)

status
閉鎖    94
dtype: int64

In [24]:
# tqdmをpandasで使えるように初期化する
tqdm.pandas(desc="ジオコーディング中...")

def geocode_address(address):

    base_url = 'https://geocode.csis.u-tokyo.ac.jp/cgi-bin/simple_geocode.cgi?addr='
    
    try:
        # タイムアウトを設定し、無限に待機するのを防ぐ
        response = requests.get(base_url + address, timeout=10)
        response.raise_for_status()  
        
        # レスポンスはXML形式のため、XMLとしてパースする
        root = ET.fromstring(response.content)
        
        # <candidate> タグが存在するかチェックする
        candidate = root.find('candidate')
        if candidate is not None:
            # <longitude> と <latitude> タグから経度と緯度を取得する
            lon = candidate.find('longitude').text
            lat = candidate.find('latitude').text
            
            # APIサーバーへの負荷を考慮し、1秒間待機させる
            time.sleep(1)
            
            # 経度と緯度をfloat型に変換して返す
            return pd.Series([float(lon), float(lat)], index=['longitude', 'latitude'])
        else:
            return pd.Series([None, None], index=['longitude', 'latitude'])

    except (requests.exceptions.RequestException, IndexError, KeyError) as e:
        print(f"住所 '{address}' の処理中にエラーが発生しました: {e}")
        return pd.Series([None, None])
    
# 元のDataFrameをコピーして、新しいDataFrame firms_with_prefecture_with_geocode を作成する
firms_with_prefecture_with_geocode = firms_with_prefecture.copy()

# 'location_x'列の各住所に対してジオコーディング関数を適用し、結果をfirms_with_prefecture_with_geocodeに新しい列として追加する
# .progress_applyを使用すると進捗状況が表示される
firms_with_prefecture_with_geocode[['longitude', 'latitude']] = firms_with_prefecture_with_geocode['location'].progress_apply(geocode_address)

# 結果を表示
firms_with_prefecture_with_geocode.head()

ジオコーディング中...:   0%|          | 0/5152 [00:00<?, ?it/s]

,corporate_number,name,location,postal_code,status,update_date,founded_year,都道府県,longitude,latitude
584,3010701042353,株式会社キカリ,東京都千代田区麹町３丁目３番地８,1020083.0,NaN,2023-07-14 00:00:00+09:00,2022,東京都,139.738632,35.683769
1143,6010001207083,株式会社ＳＥＶＥＮ ＳＥＡＬＳ,神奈川県横浜市西区みなとみらい２丁目２番１号ランドマークタワー３１Ｆ,2200012.0,NaN,2023-07-14 00:00:00+09:00,2020,神奈川県,139.631454,35.455093
1257,6130005015768,一般社団法人相続なんでも相談センター,大阪府吹田市垂水町３丁目４番４号,5640062.0,NaN,2023-07-14 00:00:00+09:00,2019,大阪府,135.502136,34.763737
7398,1430001078470,株式会社スクラッチシステムマネージメンツ,北海道札幌市中央区北三条西７丁目１番地１ＳＡＫＵＲＡ−Ｎ３,600003.0,NaN,2021-01-04 00:00:00+09:00,2019,北海道,141.345428,43.064537
7506,1430001079832,株式会社ザイマックス北海道,北海道札幌市中央区北一条西３丁目２番地,600001.0,NaN,2020-01-16 00:00:00+09:00,2019,北海道,141.352249,43.061920


In [29]:
import pandas as pd
firms_with_prefecture_with_geocode = pd.read_csv(data_path / "gbizinfo_list_2025_09_21.csv", index_col=0, low_memory=False)

In [25]:
firms_with_prefecture_with_geocode[['longitude', 'latitude']].head()

,longitude,latitude
584,139.738632,35.683769
1143,139.631454,35.455093
1257,135.502136,34.763737
7398,141.345428,43.064537
7506,141.352249,43.061920


In [26]:
import folium
# ヒートマップのプラグインをインポート
from folium.plugins import HeatMap

#NaNの場合値を除去し、latlng_dropna　に保存する
latlng_dropna=firms_with_prefecture_with_geocode[['latitude', 'longitude']].dropna().values.tolist()

# 東京駅の緯度と経度を直接指定
# [緯度, 経度] の順でリストを作成する
tokyo_station_coords = (35.681236, 139.767125)

# 緯度・経度を指定して地図の中心を設定する
tokyo_map = folium.Map(location=tokyo_station_coords, zoom_start=11)

# 緯度・経度と場所情報をヒートマップとしてプロット
HeatMap(latlng_dropna, radius=11, blur=10).add_to(tokyo_map)

In [27]:
tokyo_map

ここから下は別の教科書(from 講談社)で使う予定。メッシュ分析など。